The idea of the next few examples is simple: start a function that blocks for 3 seconds while animating characters in the terminal to let the user know that the program is “thinking” and not stalled.

The script makes an animated spinner displaying each character in the string "\|/-" in the same screen position

Spinner with Threads

In [9]:
import itertools
import time
from threading import Thread, Event

def spin(msg: str, done: Event) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end=' ', flush=True)
        if done.wait(.2):  
            break  
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')  

def slow() -> int:
    time.sleep(3)  
    return 42

def supervisor() -> int:  
    done = Event()  
    spinner = Thread(target=spin, args=('thinking!', done))  
    print(f'spinner object: {spinner}')  
    spinner.start()  
    result = slow()  
    done.set()  
    spinner.join()  
    return result

def main() -> None:
    result = supervisor()  
    print(f'Answer: {result}')

if __name__ == '__main__':
    main()

spinner object: <Thread(Thread-11 (spin), initial)>
Answer: 42  


Spinner with Processes


In [ ]:
import itertools
import time
from multiprocessing import Process, Event
from multiprocessing import synchronize

def spin(msg: str, done: synchronize.Event) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end=' ', flush=True)
        if done.wait(.2):  
            break  
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')  

def slow() -> int:
    time.sleep(3)  
    return 42

def supervisor() -> int:
    done = Event()
    spinner = Process(target=spin, 
                      args=('thinking!', done))
    print(f'spinner object: {spinner}')  
    spinner.start()
    result = slow()
    done.set()
    spinner.join()
    return result

def main() -> None:
    result = supervisor()  
    print(f'Answer: {result}')

if __name__ == '__main__':
    main()

Spinner with Coroutines

In [16]:
import asyncio
import itertools
import nest_asyncio

nest_asyncio.apply()

async def spin(msg: str) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, flush=True, end='')
        try:
            await asyncio.sleep(.1)
        except asyncio.CancelledError:
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')

async def slow() -> int:
    await asyncio.sleep(3)
    return 42

def main() -> None:
    result = asyncio.run(supervisor())
    print(f'Answer: {result}')

async def supervisor() -> int:
    spinner = asyncio.create_task(spin('thinking!'))
    print(f'spinner object: {spinner}')
    result = await slow()
    spinner.cancel()
    return result

if __name__ == '__main__':
    main()

spinner object: <Task pending name='Task-10' coro=<spin() running at /var/folders/5z/mztd4xyx4639f9nm6dyjxkpm0000gp/T/ipykernel_12312/287575052.py:7>>
| thinking!Answer: 42
